# Pytorch Transformer 구현
## nn.Transformer 모듈을 이용하는 Seq2Seq 모델 학습
### NLP에서 큰 영향을 준 모듈

In [32]:
# 언어 모델링 과제에 대해 학습
## 언어 모델링 과제 = 주어진 단어가 다음에 이어지는 단어 시퀀스를 따를 가능성 (Likelihood)에 대한 확률을 할당하는 것
## token(seqence) -> embedding -> positional encoding -> 미래 token에 대해서는 몰라야하기 때문에 masking
### 최종적으로 얻는 단어는 log-softmax로 Linear 레이어로 전달

In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()


    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Arguemnts:
            src : Tensor, shape ``[seq_len, batch_size] ``
            src_mask : Tensor, shape ``[seq_len, seq_len] ``

        Returns :
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """

        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src) # positional encoder
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output
    
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """ Generates an upper-triangular matrix of ``-inf``, with zeros on ``diag``."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

print(generate_square_subsequent_mask(5))


tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])


In [34]:
# positional embedding 함수, cosin, sin 함수 사용

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float=0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term) # 짝수는 sin 주파수
        pe[:, 0, 1::2] = torch.cos(position * div_term)  # 홀수는 cos 주파수
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x : Tensor, shape ``[seq_len, batch_size, embedding_dim] ``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


In [35]:
# pip install torchdata

# vocab 객체는 훈련 데이터셋에 의해 만들어지고,
# token을 텐서로 수치화 하는데 사용됨
## rare token(보기 드문 토큰)은 <unk>로 표현됨

# batchify() 함수 : 데이터를 batch_size 컬럼들로 정렬
## 만약, batch_size 컬럼으로 나눠 떨어지지 않으면, 데이터를 잘라서 맞춤
## ex) A ~ Z(26자) -> batch_size = 4 -> [[A, B, * 6],[6],[6],[6]] 나눠짐

from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-v1")

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = dataset['train']['text']
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter: # (text)
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter) -> Tensor:
    """ Converts raw text into a float Tensor. """
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    # cat -> tensor 연결
    # filter -> lambda 함수에 해당하는 것만 필터링
    # t.numel()은 tensor의 요소가 몇 개인지, 하나 이상인 텐서만 선택
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


train_iter, val_iter, test_iter = dataset['train']['text'], dataset['validation']['text'], dataset['test']['text']
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [36]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """
    Divides the data into ``bsz`` separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Arguments:
        data : Tensor, shape ``[N]``
        bsz : int, batch size

    Returns:
        Tensor of shape ``[N // bsz, bsz]```
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20 # batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size) # shapep [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [37]:
## get_batch : input-target sequence pair를 생성
## source data를 bptt 길이를 가진 덩어리로 세분화

bptt = 35

def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Arguments:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple ``(data, target)``, where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len] # 앞에 i부터 ~ seq_len 만큼 자름
    target = source[i+1:i+1+seq_len].reshape(-1) # i+1부터 ~ seq_len + 1 만큼 예측
    return data, target

In [38]:
ntokens = len(vocab) # 단어 사전(어휘집)의 크기
emsize = 200 # 임베딩 차원
d_hid = 200 # ``nn.TransformerEncoder`` 에서 피드포워드 네트워크(feedforward network) 모델의 차원
nlayers = 2 # ``nn.TransformerEncoder`` 내부의 nn.TransformerEncoderLayer 개수 encoder 2계층
nhead = 2 # ``nn.MultiheadAttention`` 의 헤드 개수 # 2-heads attentions
dropout = 0.2 # 드랍아웃(dropout) 확률
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [39]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # 학습률(learning rate)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
# step size마다 gamma 비율로 lr을 감소시킴, step_size = 1.0

def train(model: nn.Module) -> None:
    model.train()  # 학습 모드 시작
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # 마지막 배치에만 적용(다른 배치는 bptt길이)
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        # 기울기 폭발 방지
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # 평가 모드 시작
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [40]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 22.69 | loss  8.23 | ppl  3749.85
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 15.44 | loss  6.92 | ppl  1013.94
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 14.61 | loss  6.46 | ppl   639.96
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 12.64 | loss  6.32 | ppl   556.14
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 12.25 | loss  6.20 | ppl   492.59
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 14.27 | loss  6.16 | ppl   473.87
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 13.26 | loss  6.12 | ppl   453.83
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 14.13 | loss  6.11 | ppl   448.82
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 13.32 | loss  6.04 | ppl   418.16
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 12.95 | loss  6.03 | ppl   415.20
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 12.43 | loss  5.90 | ppl   365.99
| epoch   1 |  2400/ 

In [41]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.46 | test ppl   234.87
